In [1]:
import os
import re
from selenium import webdriver

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

In [3]:
driver.get("https://www.google.com")
print(f"Page title: {driver.title}")

Page title: Google


In [7]:
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import numpy as np
from selenium.webdriver.common.by import By
import pandas as pd
import re

list_unis_ca_B = """
University of New Brunswick
University of Northern British Colombia
University of Ontario Institute of Technology
University of Regina
University of Sherbrooke
    """

root_url = "https://scholar.google.com/"
last_citation_boundary = 300
data = []

driver.get(root_url)


def extract_numbers_from_string(input_string):
    # Regular expression pattern to match numbers (including integers and floats)
    pattern = r'[-+]?\d*\.\d+|\d+'

    # Using re.findall() to find all matches of the pattern in the input string
    numbers = re.findall(pattern, input_string)

    # Converting the matched strings to actual numbers (integers or floats)
    numbers = [int(num) if '.' not in num else float(num) for num in numbers]

    return numbers



def open_university_in_scholar(university_name,driver=driver):
    driver.get(root_url)
    time.sleep(2.5)
    while ("Our systems have detected unusual traffic" in driver.page_source):
        print("captcha")
        clickable_element = driver.find_element(By.CLASS_NAME, 'g-recaptcha')
        clickable_element.click()
        time.sleep(10)
        print("clicking_capthcha")


    time.sleep(1.5)
    driver.get(root_url)
    search_element = driver.find_element(By.ID,"gs_hdr_tsi")
    search_element.send_keys(university_name)
    search_element.send_keys(Keys.ENTER)
    time.sleep(1.3)
    university_element = driver.find_element(By.CLASS_NAME,"gs_r")
    university_element.find_element(By.XPATH,"//div[@class='gs_ob_inst_r']/a").click()
    time.sleep(1.82)



for uni in list_unis_ca_B.split("\n"):
    university = uni.split(",")[0]
    university = university.strip()
    if len(university) < 5:
        continue
    print(university)
    open_university_in_scholar(university)
    last_citation = last_citation_boundary+1
    while last_citation > last_citation_boundary :
        users = driver.find_elements(By.CLASS_NAME,'gsc_1usr')
        for user in users:
            html_data = user.get_attribute("outerHTML")
            soup = BeautifulSoup(html_data, 'html.parser')
            name_tag = soup.select_one('h3.gs_ai_name a')
            name = name_tag.text.strip()
            link = root_url + name_tag['href']
            data.append({'Name': name, 'Link': link})

        last_citation = int(extract_numbers_from_string(users[-1].find_element(By.CLASS_NAME,'gs_ai_cby').text)[0])
        element_xpath = "//button[@aria-label='Next']"
        driver.find_element(By.CLASS_NAME,"gsc_pgn").find_element(By.CSS_SELECTOR,"button.gsc_pgn_pnx").click()
        print(last_citation)
        time.sleep(np.random.random()*4+3)
        df = pd.DataFrame(data)
        df.to_csv(f"ca_B/Canada_B_2.csv")

University of New Brunswick
13130
9336
6880
5774
4029
3203
2712
2405
2075
1780
1598
1372
1203
1060
805
731
641
560
485
420
378
330
292
University of Northern British Colombia
6834
3220
2149
1243
879
689
526
351
249
University of Ontario Institute of Technology
3851
2318
1517
918
597
266
University of Regina
10127
6162
3674
2731
2484
2094
1804
1435
1223
1048
871
778
678
573
514
449
359
298
University of Sherbrooke
22635
13958
10921
8486
7457
6588
5997
5125
4698
4172
3762
3434
3258
2969
2813
2615
2432
2309
2052
1893
1826
1572
1439
1347
1287
1192
1116
1042
978
919
880
822
777
672
633
602
567
523
507
457
420
399
389
365
341
314
292


In [ ]:
driver.page_source